In [1]:
import torch
import torch.nn as nn
import numpy as np
from option.coordinet_option import CoordiNetOption
from utils.utils import *
from model.coordinet import CoordiNetSystem
from dataset.cambridge import CambridgeDataset
from dataset.seven_scenes import SevenScenesDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys
import pickle
import matplotlib.pyplot as plt

In [2]:
opt_path='/root/coordinet/runs/coordinet/church_train_synthesis2/opt.pkl'
with open(opt_path, 'rb') as f:
    opt = pickle.load(f)
    opt.reshape_size=320
    opt.crop_size=300
    opt.fixed_weight=False
    opt.backbone='efficientnet'
    opt.chunk=8
# dataset
train_dataset = CambridgeDataset(opt.data_root_dir, opt.scene, 'train',opt.reshape_size, opt.crop_size)
# train_dataset= SevenScenesDataset(opt.data_root_dir, opt.scene, 'train', opt.reshape_size, opt.crop_size)
valid_dataset = CambridgeDataset(opt.data_root_dir, opt.scene, 'valid',opt.reshape_size, opt.crop_size)
# valid_dataset= SevenScenesDataset(opt.data_root_dir, opt.scene, 'valid', opt.reshape_size, opt.crop_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=opt.chunk, num_workers=16, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=opt.chunk, num_workers=16, shuffle=False)

load data of scene "fire", split: train
load data of scene "fire", split: valid


In [3]:
# init model
ckpt_path = '/root/coordinet/runs/coordinet/church_train_synthesis2/ckpt_epoch133_steps1006_angle5.1000_t1.1074.pkl'
opt.var_min=[0.25,0.25,0.25,0.0314]
coordinet = CoordiNetSystem(opt.feature_dim,opt.W,opt.loss_type,opt.learn_beta,opt.var_min,opt.fixed_weight,opt.backbone,opt.backbone_path)
ckpt = torch.load(ckpt_path)
coordinet.load_state_dict(ckpt['coordinet'])
print('load model from {}'.format(ckpt_path))
coordinet = coordinet.cuda()
coordinet.eval();

Loaded pretrained weights for efficientnet-b3
load model from /root/coordinet/runs/coordinet/fire/ckpt_epoch112_steps100_angle17.2935_t0.3746.pkl


In [4]:
# valid: center crop from test set
t_diffs = []
angle_diffs = []
ts=[]
for imgs, poses in tqdm(valid_dataloader, total=len(valid_dataloader), file=sys.stdout):
    imgs = imgs.cuda()
    poses = poses.cuda()
    poses[:,:,-1]=poses[:,:,-1]
    with torch.no_grad():
        t, q, _, losses = coordinet.forward(imgs, cal_loss=True, y_pose=poses)
    ts.append(t.cpu().numpy())
    t_diffs.append(losses['t_diff'].cpu().numpy())
    angle_diffs.append(losses['angle_diff'].cpu().numpy())
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs) / np.pi * 180
angle_diff = np.median(angle_diffs)
t_diff = np.median(np.hstack(t_diffs))
print('[valid][median] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))

ts=np.vstack(ts)
# save
with open('./valid_median_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        GT_t=valid_dataset.test_poses[i][:3,3]
        t=ts[i]
        f.write('%.4fm %.4f° [GT:(%.3f,%.3f,%.3f)] [Re:(%.3f,%.3f,%.3f)]\n' % (t_diffs[i], angle_diffs[i],*list(GT_t),*list(t)))

  3%|▎         | 3/100 [00:04<01:52,  1.16s/it]

/root/miniconda3/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


100%|██████████| 100/100 [00:09<00:00, 10.47it/s]
[valid] angle_diff:17.2935°, t_diff:0.3746m


In [5]:
# test: uniform num crops from test set
t_diffs = []
angle_diffs = []
ts=[]
num_crops=128
print('every test image random uniform crop %d times' % num_crops)
for i in tqdm(range(len(valid_dataset.test_paths))):
    imgs,poses=valid_dataset.uniform_crop(valid_dataset.test_paths[i],num_crops,valid_dataset.test_poses[i])
    it,it_diffs,iangle_diffs=[],[],[]
    for j in range(0,len(imgs),opt.chunk):
        img,pose=imgs[j:j+opt.chunk],poses[j:j+opt.chunk]
        img,pose=img.cuda(),pose.cuda()
        with torch.no_grad():
            t, q, _, losses = coordinet.forward(img, cal_loss=True, y_pose=pose)
        it.append(t.cpu().numpy())
        it_diffs.append(losses['t_diff'].cpu().numpy())
        iangle_diffs.append(losses['angle_diff'].cpu().numpy())
    it = np.vstack(it)
    it_diffs = np.hstack(it_diffs)
    iangle_diffs = np.hstack(iangle_diffs) / np.pi * 180
    ts.append(np.mean(it,axis=0))
    t_diffs.append(np.mean(it_diffs))
    angle_diffs.append(np.mean(iangle_diffs))
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs)
t_diff=np.median(t_diffs)
angle_diff=np.median(angle_diffs)
print('[test][median] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))
ts=np.vstack(ts)
# save
with open('./test_median_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        GT_t=valid_dataset.test_poses[i][:3,3]
        t=ts[i]
        f.write('%.4fm %.4f° [GT:(%.3f,%.3f,%.3f)] [Re:(%.3f,%.3f,%.3f)]\n' % (t_diffs[i], angle_diffs[i],*list(GT_t),*list(t)))

100%|██████████| 2000/2000 [10:12<00:00,  3.26it/s]

[test] angle_diff:17.1682°, t_diff:0.5989m


In [ ]:
# train
coordinet.eval()
t_diffs = []
angle_diffs = []
for imgs, poses in tqdm(train_dataloader, total=len(train_dataloader), file=sys.stdout):
    imgs = imgs.cuda()
    poses = poses.cuda()
    with torch.no_grad():
        t, q, _, losses = coordinet.forward(imgs, cal_loss=True, y_pose=poses)
    t_diffs.append(losses['t_diff'].cpu().numpy())
    angle_diffs.append(losses['angle_diff'].cpu().numpy())
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs) / np.pi * 180
angle_diff = np.mean(angle_diffs)
t_diff = np.mean(t_diffs)
print('[train][mean] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))
# save
with open('./train_mean_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        f.write('%.4fm %.4f°\n' % (t_diffs[i], angle_diffs[i]))